In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
outputs = ['.\\Outputs', "TAZ.gdb"]

if not os.path.exists(outputs[0]):
    os.makedirs(outputs[0])

gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])


In [4]:
hh = pd.DataFrame.spatial.from_featureclass(r'E:\Tasks\Box-Elder-County-Split\Outputs\TAZ_UPDATED.gdb\households')
hh.head(2)

,OBJECTID,CityArea,SUBAREAID,ModelArea,CO_TAZID,DEVACRES,CO_FIPS,CO_NAME,SMALLAREA,TAZID,RELEASE,SECategory,YEAR2019,YEAR2020,YEAR2021,YEAR2022,YEAR2023,YEAR2024,YEAR2025,YEAR2026,YEAR2027,YEAR2028,YEAR2029,YEAR2030,YEAR2031,YEAR2032,YEAR2033,YEAR2034,YEAR2035,YEAR2036,YEAR2037,YEAR2038,YEAR2039,YEAR2040,YEAR2041,YEAR2042,YEAR2043,YEAR2044,YEAR2045,YEAR2046,YEAR2047,YEAR2048,YEAR2049,YEAR2050,CH19TO50,CH19TO23,CH19TO25,CH20TO30,CH23TO28,CH28TO32,CH30TO40,CH32TO40,CH32TO42,CH40TO50,CH42TO50,YEAR2019D,YEAR2020D,YEAR2023D,YEAR2024D,YEAR2028D,YEAR2030D,YEAR2032D,YEAR2040D,YEAR2042D,YEAR2050D,CH19TO50D,CH19TO23D,CH19TO25D,CH20TO30D,CH23TO28D,CH28TO32D,CH30TO40D,CH32TO42D,CH40TO50D,CH42TO50D,MOREINFO,SHAPE
0,1,Cornish,2,Cache Travel Demand Model,50001,490.0959,5,CACHE,,7092,<NA>,TOTHH,5.8,5.9,6.1,6.3,6.5,6.7,6.8,7.0,7.1,7.3,7.4,7.5,7.6,7.6,7.7,7.8,7.8,7.9,7.9,8.0,8.0,8.1,8.4,8.6,8.9,9.1,9.4,9.6,9.9,10.1,10.3,10.6,4.8,0.7,1.0,1.6,0.8,0.3,0.6,0.5,1.0,2.5,2.0,0.012,0.012,0.013,0.014,0.015,0.015,0.016,0.017,0.018,0.022,0.01,0.001,0.002,0.003,0.002,0.001,0.001,0.002,0.005,0.004,<NA>,"{""rings"": [[[-12463155.3059, 5160740.102399997..."
1,2,Cornish,2,Cache Travel Demand Model,50002,564.3828,5,CACHE,,7093,<NA>,TOTHH,24.3,24.8,25.1,25.4,25.8,26.1,26.4,26.7,26.9,27.1,27.3,27.5,27.6,27.7,27.8,27.9,28.0,28.1,28.2,28.3,28.3,28.4,28.8,29.2,29.5,29.9,30.2,30.6,30.9,31.2,31.5,31.8,7.5,1.5,2.1,2.7,1.3,0.6,0.9,0.7,1.5,3.4,2.6,0.043,0.044,0.046,0.046,0.048,0.049,0.049,0.05,0.052,0.056,0.013,0.003,0.004,0.005,0.002,0.001,0.002,0.003,0.006,0.005,<NA>,"{""rings"": [[[-12462732.2341, 5160744.644000001..."


In [5]:
jobs = pd.DataFrame.spatial.from_featureclass(r'E:\Tasks\Box-Elder-County-Split\Outputs\TAZ_UPDATED.gdb\typical_jobs')
jobs.head(2)

,OBJECTID,CityArea,SUBAREAID,ModelArea,CO_TAZID,DEVACRES,CO_FIPS,CO_NAME,SMALLAREA,TAZID,RELEASE,SECategory,YEAR2019,YEAR2020,YEAR2021,YEAR2022,YEAR2023,YEAR2024,YEAR2025,YEAR2026,YEAR2027,YEAR2028,YEAR2029,YEAR2030,YEAR2031,YEAR2032,YEAR2033,YEAR2034,YEAR2035,YEAR2036,YEAR2037,YEAR2038,YEAR2039,YEAR2040,YEAR2041,YEAR2042,YEAR2043,YEAR2044,YEAR2045,YEAR2046,YEAR2047,YEAR2048,YEAR2049,YEAR2050,CH19TO50,CH19TO23,CH19TO25,CH20TO30,CH23TO28,CH28TO32,CH30TO40,CH32TO40,CH32TO42,CH40TO50,CH42TO50,YEAR2019D,YEAR2020D,YEAR2023D,YEAR2024D,YEAR2028D,YEAR2030D,YEAR2032D,YEAR2040D,YEAR2042D,YEAR2050D,CH19TO50D,CH19TO23D,CH19TO25D,CH20TO30D,CH23TO28D,CH28TO32D,CH30TO40D,CH32TO42D,CH40TO50D,CH42TO50D,MOREINFO,SHAPE
0,1,Cornish,2,Cache Travel Demand Model,50001,490.0959,5,CACHE,,7092,<NA>,TYPEMP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<NA>,"{""rings"": [[[-12463155.3059, 5160740.102399997..."
1,2,Cornish,2,Cache Travel Demand Model,50002,564.3828,5,CACHE,,7093,<NA>,TYPEMP,7.4,7.3,7.8,7.9,8.0,8.3,8.4,8.4,8.6,8.7,8.9,9.0,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.9,9.9,10.1,10.2,10.3,10.3,10.3,10.4,10.4,10.6,10.7,10.7,3.3,0.6,1.0,1.7,0.7,0.4,0.9,0.8,1.1,0.8,0.5,0.013,0.013,0.014,0.015,0.015,0.016,0.016,0.018,0.018,0.019,0.006,0.001,0.002,0.003,0.001,0.001,0.002,0.002,0.001,0.001,<NA>,"{""rings"": [[[-12462732.2341, 5160744.644000001..."


In [6]:
hh2 = hh[['CO_TAZID','YEAR2019', 'YEAR2020', 'YEAR2023','YEAR2024', 'YEAR2025', 'YEAR2030', 'YEAR2032', 'YEAR2035', 'YEAR2040', 'YEAR2042', 'YEAR2045', 'YEAR2050']].copy()
new_hh_columns = ['HH_'+ x[4:] if x.startswith('YEAR') else x for x in hh2.columns]
hh2.columns = new_hh_columns

jobs2 = jobs[['CO_TAZID','YEAR2019', 'YEAR2020', 'YEAR2023','YEAR2024', 'YEAR2025', 'YEAR2030', 'YEAR2032', 'YEAR2035', 'YEAR2040', 'YEAR2042', 'YEAR2045', 'YEAR2050']].copy()
new_jobs_columns = ['JOB_'+ x[4:] if x.startswith('YEAR') else x for x in jobs2.columns]
jobs2.columns = new_jobs_columns

In [8]:
template = hh[hh['SUBAREAID'] ==1].copy()
template = template[['CO_TAZID','TAZID', 'DEVACRES', 'SHAPE']].copy()
template = template.merge(hh2, on='CO_TAZID', how='left').merge(jobs2, on='CO_TAZID', how='left')
template.spatial.to_featureclass(location=os.path.join(gdb, 'ATO'),sanitize_columns=False)

'e:\\Tasks\\Box-Elder-County-Split\\Outputs\\TAZ.gdb\\ATO'